In [1]:
import pandas as pd
import datetime
import time
import pickle
import os
import numpy as np
from pyvital2 import vitaldb


# ECG 250Hz pickle로 저장하기

In [13]:
! rm -rf 'vital_data/PPG_100Hz_ECG_100Hz_pacu_5min'

In [6]:
df_match = pd.read_csv('ftn+_age_match_20201224.csv',sep=',')
df_match['NRS_time']=pd.to_datetime(df_match['NRS_time'])
df_match = df_match.rename(columns={'Unnamed: 0':'index'})
#df_match.reset_index(inplace=True, drop=True)

print(len(df_match))
'''vital -> csv
df_match['path']=[f_name.split('.')[0]+".csv" for f_name in df_match['path'].values.flatten()] '''

srate= 100
for i in range(2500,0,-1):
    filename = str(df_match.loc[i,'Value'])+','+str(df_match.loc[i,'index'])+','+ df_match.loc[i,'path']
    if not os.path.exists('vital_data/PPG_100Hz_ECG_100Hz_pacu_5min/'+filename):
        #try: 
        rec_path = 'vital_data/pacu_ecg_pleth/pacu_ecg_pleth/'+df_match.loc[i,'path']
        #print(rec_path)
        #print('1')
        vfile_ecg = vitaldb.vital_recs_time(rec_path,['SNUADCW/ECG_II'],interval=0.01).flatten()
        vfile_ppg = vitaldb.vital_recs_time(rec_path,['SNUADCW/PLETH'],interval=0.01).flatten()

        #print(vfile)
        if len(vfile_ecg[0])==0 | len(vfile_ppg[0])==0:
            print('index ',df_match.loc[i,'index'],' OPID ',df_match.loc[i,'opid'],'empty vital file')
        else:
            Abs_time = [vfile_ecg[1]+datetime.timedelta(hours=9)+datetime.timedelta(seconds=i/srate) for i in range(len(vfile_ecg[0]))]
            #print(vfile[1])
            dics = {'Abs_time':Abs_time,'PPG':vfile_ppg[0],'ECG':vfile_ecg[0]}
            df_vital = pd.DataFrame(data=dics)
            end_idx_list = df_vital[(df_vital['Abs_time'] > df_match.loc[i,'NRS_time'] - datetime.timedelta(seconds=1)) & (df_vital['Abs_time'] < df_match.loc[i,'NRS_time'] + datetime.timedelta(seconds=1))].index.tolist()

            if len(end_idx_list)==0:
                print('index ',i,' OPID ',df_match.loc[i,'opid']," no vital data at NRS time")

            else:
                end_idx=end_idx_list[0]+1
                start_idx= end_idx - 5 * 60 * srate

                ext = df_vital[start_idx:end_idx]
                extr=ext.dropna(subset=['ECG'])
                extr=extr.drop_duplicates(subset=['ECG'])

                if len(extr)>1:
                    if start_idx <0 :
                        fit = (-1)*end_idx 
                        extraction = pd.DataFrame(index=range(5*60*srate),columns=['Abs_time','PPG','ECG'])
                        extraction[fit:]=ext

                        print(extr)
                        print(df_match.loc[i,'path'])

                    else:
                        extraction = ext
                    with open('vital_data/PPG_100Hz_ECG_100Hz_pacu_5min/'+filename,'wb') as f:
                        pickle.dump(extraction, f)
                    print(i,'...done')
        #except:
            #print('error: ',i,df_match.loc[i,'opid'],df_match.loc[i,'path'])

10156
2499 ...done
2498 ...done
2497 ...done
2495 ...done
2494 ...done
2493 ...done
2492 ...done
2490 ...done
2489 ...done
2488 ...done
2487 ...done
2486 ...done
2485 ...done
2484 ...done
2483 ...done
2482 ...done
2481 ...done
2480 ...done
2479 ...done
2478 ...done
2477 ...done
2476 ...done
2475 ...done
2474 ...done
2473 ...done
2472 ...done
2471 ...done
2470 ...done
2469 ...done
2468 ...done
2467 ...done
2466 ...done
2464 ...done
2463 ...done
2462 ...done
2461 ...done
2460 ...done
2459 ...done
2458 ...done
2457 ...done
2456 ...done
2454 ...done
2452 ...done
2451 ...done
2450 ...done
2449 ...done
2448 ...done
2446 ...done
2444 ...done
2443 ...done
2442 ...done
2441 ...done
2440 ...done
2439 ...done
2438 ...done
2437 ...done
2436 ...done
2434 ...done
2433 ...done
2432 ...done
2431 ...done
2430 ...done
2429 ...done
2428 ...done
2427 ...done
2425 ...done
2424 ...done
2423 ...done
2422 ...done
2421 ...done
2420 ...done
2419 ...done
2418 ...done
2417 ...done
2416 ...done
2415 ...done
2414 .

KeyboardInterrupt: 

# NIBP PACU pickle로 저장하기

In [4]:
import pandas as pd
import datetime
import time
import pickle
import os
import numpy as np
from pyvital2 import vitaldb

df_match = pd.read_csv('match_drug_arr_exclusion_20201110.csv',sep=',')
df_match['NRS_time']=pd.to_datetime(df_match['NRS_time'])
df_IDpath = pd.read_csv('opid_fileid.csv')
#df_match['path']=[f_name.split('.')[0]+".csv" for f_name in df_match['path'].values.flatten()]

for j in range(len(df_IDpath)):
    #try: 
    #opid match
    idx_list = df_match[df_match['opid']==df_IDpath.loc[j,'opid']].index.tolist()
    i = min(idx_list)

    rec_path = 'preop_vital/preop_vital/' + df_IDpath.loc[j,'fileid']
    df_IDpath.loc[j, 'file_path'] = df_match.loc[i, 'path']
    df_IDpath.loc[j, 'NRS'] = df_match.loc[i, 'Value']
    df_IDpath.loc[j, 'NRS_time'] = df_match.loc[i, 'NRS_time']
print(df_IDpath)
print(df_match)

           opid                  fileid                    file_path   NRS  \
0     190408044  A2_190408_103443.vital  PACU1_1_190408_115004.vital   6.0   
1     190408106  E3_190408_180036.vital  PACU1_1_190408_202459.vital   7.0   
2     190409012  A2_190409_073654.vital  PACU1_2_190409_110712.vital   4.0   
3     190409044  D5_190409_101311.vital  PACU1_3_190409_114333.vital   6.0   
4     190409057  F4_190409_105106.vital  PACU1_1_190409_133948.vital   8.0   
...         ...                     ...                          ...   ...   
4109  201030106  R9_201030_145209.vital  PACU1_1_201030_161119.vital   6.0   
4110  201030057  E6_201030_131959.vital  PACU1_3_201030_195639.vital   7.0   
4111  201030082  R2_201030_185910.vital  PACU1_5_201030_203752.vital   6.0   
4112  201030023  C4_201030_185047.vital  PACU1_1_201030_211816.vital   6.0   
4113  201031004  R9_201031_084856.vital  PACU1_3_201031_131135.vital  10.0   

                NRS_time  
0    2019-04-08 12:00:00  
1    2019

In [ ]:
import pandas as pd
import datetime
import time
import pickle
import os
import numpy as np
from pyvital2 import vitaldb

'''vital -> csv
df_match['path']=[f_name.split('.')[0]+".csv" for f_name in df_match['path'].values.flatten()] '''

srate= 250
for i in range(len(df_match)):
    bedno = df_match.loc[i,'path'].split('_')[1]
    if int(bedno) <4:
        filename = str(df_match.loc[i,'Value'])+','+str(i)+','+ df_match.loc[i,'path']
        if not os.path.exists('vital_data/NIBP_250Hz_pacu_2min_pickle/'+filename):
            rec_path = 'PACU_vital/PACU1_1-3/'+df_match.loc[i,'path']
            vfile = vitaldb.vital_recs_time(rec_path,['Solar8000/NIBP_MBP'],interval=0.004).flatten()
            #print(vfile)
            if len(vfile[0])==0:
                print('index ',i,' OPID ',df_match.loc[i,'opid'],'empty NIBP')
            else:
                Abs_time = [vfile[1]+datetime.timedelta(hours=9)+datetime.timedelta(seconds=i/srate) for i in range(len(vfile[0]))]
                #print(vfile[1])
                dics = {'Abs_time':Abs_time,'NIBP':vfile[0]}
                df_NIBP = pd.DataFrame(data=dics)
                end_idx_list = df_NIBP[(df_NIBP['Abs_time'] > df_match.loc[i,'NRS_time'] - datetime.timedelta(seconds=1)) & (df_NIBP['Abs_time'] < df_match.loc[i,'NRS_time'] + datetime.timedelta(seconds=1))].index.tolist()

                if len(end_idx_list)==0:
                    print('index ',i,' OPID ',df_match.loc[i,'opid']," no vital data at NRS time")                  
                else:
                    end_idx=end_idx_list[0]
                    start_idx= max(0,end_idx - 2 * 60 * srate)
                    extraction = df_NIBP.loc[start_idx:end_idx]
                    with open('vital_data/NIBP_250Hz_pacu_2min_pickle/'+filename,'wb') as f:
                        pickle.dump(extraction, f)
                        
#이건 2분보다 짧아도 저장함

# NIBP PACU pickle로 저장하기

In [ ]:
import pandas as pd
import pickle
import datetime
import time
from pyvital2 import vitaldb
import numpy as np
import os

'''vital -> csv
df_match['path']=[f_name.split('.')[0]+".csv" for f_name in df_match['path'].values.flatten()] '''

srate= 250
for i in range(len(df_IDpath)):
    filename = df_IDpath.loc[i,'file_path']
    if not os.path.exists('vital_data/NIBP_250Hz_preop_2min_pickle/'+filename):
        rec_path = 'preop_vital/preop_vital/'+ df_IDpath.loc[i,'fileid']
        vfile = vitaldb.vital_recs(rec_path,['Solar8000/NIBP_MBP'],interval=0.004).flatten()[0:2*60*250]

        #print(vfile)
        if len(vfile) ==0 :
            print('index ',i,' OPID ',df_IDpath.loc[i,'opid'],'empty NIBP')
        else:
            dics = {'NIBP':vfile}
            df_NIBP = pd.DataFrame(data=dics)

            extraction = df_NIBP
            with open('vital_data/NIBP_250Hz_preop_2min_pickle/'+filename,'wb') as f:
                pickle.dump(extraction, f)
            print(i,'/',len(df_IDpath),' save')

In [1]:
import pandas as pd
import datetime, time, pickle, os, vitaldb


df_match = pd.read_csv('ftn+_age_match_20201224-Copy1.csv',sep=',')
df_match['NRS_time']=pd.to_datetime(df_match['NRS_time'])
df_match = df_match.rename(columns={'Unnamed: 0':'index'})
#df_match.reset_index(inplace=True, drop=True)
#print(len(df_match))


error_list = []
srate= 100
for i in range(7500,len(df_match)):
    filename = str(df_match.loc[i,'Value'])+','+str(df_match.loc[i,'index'])+','+ df_match.loc[i,'path']
    vital_path = '../../../cranberry2/Preprocessing/vital_data/PPG_100Hz_ECG_100Hz_pacu_5min/'+filename
    if os.path.exists(vital_path):
        print(i, end=' ')
        rec_path = '../../../cranberry2/Preprocessing/vital_data/pacu_full_vital/'+df_match.loc[i,'path']
        #print(rec_path)
        
        
        # vitaldb 에러에 해당되는 케이스
        df_vital = pickle.load(open(vital_path,'rb'))
        if np.nanmean(df_vital['ECG'].tolist()) > -4 and np.nanmean(df_vital['PPG'].tolist()) > - 100:
            print('already done')
            continue
        
        error_list.append(df_match.loc[i,'path'])
        print(df_match.loc[i,'path'], end='... ')
        vfile_ecg = vitaldb.vital_recs_time(rec_path,['SNUADCW/ECG_II'],interval=0.01).flatten()
        vfile_ppg = vitaldb.vital_recs_time(rec_path,['SNUADCW/PLETH'],interval=0.01).flatten()
        
        
        #print(vfile)
        # vital이 비어있는 경우
        if len(vfile_ecg[0])==0 | len(vfile_ppg[0])==0:
            print('index ',df_match.loc[i,'index'],' OPID ',df_match.loc[i,'opid'],'empty vital file')
            continue
        
        
        Abs_time = [vfile_ecg[1]+datetime.timedelta(hours=9)+datetime.timedelta(seconds=i/srate) for i in range(len(vfile_ecg[0]))]
        #print(vfile[1])
        dics = {'Abs_time':Abs_time,'PPG':vfile_ppg[0],'ECG':vfile_ecg[0]}
        df_vital = pd.DataFrame(data=dics)
        end_idx_list = df_vital[(df_vital['Abs_time'] > df_match.loc[i,'NRS_time'] - datetime.timedelta(seconds=1)) & (df_vital['Abs_time'] < df_match.loc[i,'NRS_time'] + datetime.timedelta(seconds=1))].index.tolist()

        if len(end_idx_list)==0:
            print('index ',i,' OPID ',df_match.loc[i,'opid']," no vital data at NRS time")

        else:
            end_idx=end_idx_list[0]+1
            start_idx= end_idx - 5 * 60 * srate
            start_idx_n = max(0,start_idx)


            ext = df_vital[start_idx_n:end_idx]
            extr=ext.dropna(subset=['ECG'])
            extr=extr.drop_duplicates(subset=['ECG'])

            if len(extr)>1:
                if start_idx <0 :
                    fit = (-1)*end_idx 
                    extraction = pd.DataFrame(index=range(5*60*srate),columns=['Abs_time','PPG','ECG'])
                    extraction[fit:]=ext


                else:
                    extraction = ext
                    
                with open('../../../cranberry2/Preprocessing/vital_data/PPG_100Hz_ECG_100Hz_pacu_5min/'+filename,'wb') as f:
                    pickle.dump(extraction, f)
                print('...done')

        #except:
            #print('error: ',i,df_match.loc[i,'opid'],df_match.loc[i,'path'])

7500 already done
7501 already done
7502 already done
7503 already done
7504 already done
7505 already done
7506 already done
7507 already done
7508 already done
7509 already done
7510 already done
7511 already done
7512 already done
7513 already done
7514 PACU1_2_200921_171750.vital... 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: RuntimeWarning: Mean of empty slice


...done
7515 already done
7516 already done
7517 already done
7518 already done
7519 already done
7520 already done
7521 already done
7522 already done
7523 already done
7524 already done
7525 already done
7526 already done
7527 already done
7528 already done
7529 already done
7530 already done
7531 already done
7532 already done
7533 already done
7534 already done
7535 already done
7536 already done
7537 already done
7538 already done
7539 already done
7540 already done
7541 already done
7542 already done
7543 already done
7544 already done
7545 already done
7546 PACU1_5_200923_085343.vital... 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: RuntimeWarning: Mean of empty slice


...done
7547 already done
7548 already done
7549 already done
7550 already done
7551 already done
7552 already done
7553 already done
7554 already done
7555 already done
7556 already done
7557 already done
7558 already done
7559 already done
7560 already done
7561 already done
7562 already done
7563 already done
7564 already done
7565 already done
7566 already done
7567 already done
7568 already done
7569 already done
7570 already done
7571 already done
7572 already done
7573 already done
7574 already done
7575 already done
7576 already done
7577 already done
7578 already done
7579 already done
7580 already done
7581 already done
7582 already done
7583 already done
7584 already done
7585 already done
7586 PACU1_7_200923_173426.vital... 

...

...done
9055 already done
9056 already done
9057 already done
9058 already done
9059 already done
9060 already done
9061 already done
9062 already done
9063 already done
9064 already done
9065 already done
9066 already done
9067 already done
9068 already done
9069 already done
9070 already done
9071 already done
9072 already done
9073 already done
9074 already done
9075 already done
9076 already done
9077 already done
9078 already done
9079 already done
9080 already done
9081 already done
9082 already done
9083 already done
9084 already done
9085 already done
9086 already done
9087 already done
9088 already done
9089 already done
9090 already done
9091 already done
9092 already done
9093 already done
9094 already done
9095 already done
9096 already done
9097 already done
9098 already done
9099 already done
9100 already done
9101 already done
9102 already done
9103 already done
9104 already done
9105 already done
9106 already done
9107 already done
9108 already done
9109 already done
91

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: RuntimeWarning: Mean of empty slice


...done
9434 already done
9435 already done
9436 already done
9437 already done
9438 already done
9439 already done
9440 already done
9441 already done
9442 already done
9443 already done
9444 already done
9445 already done
9446 already done
9447 already done
9448 already done
9449 already done
9450 already done
9451 already done
9452 already done
9453 already done
9454 already done
9455 already done
9456 already done
9457 already done
9458 already done
9459 already done
9460 already done
9461 already done
9462 already done
9463 already done
9464 already done
9465 already done
9466 already done
9467 already done
9468 already done
9469 already done
9470 already done
9471 already done
9472 already done
9473 already done
9474 already done
9475 already done
9476 already done
9477 already done
9478 already done
9479 already done
9480 already done
9481 already done
9482 already done
9483 already done
9484 already done
9485 already done
9486 already done
9487 already done
9488 already done
94

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: RuntimeWarning: Mean of empty slice


...done
9721 already done
9722 already done
9723 already done
9724 already done
9725 already done
9726 already done
9727 already done
9728 already done
9729 already done
9730 already done
9731 already done
9732 already done
9733 already done
9734 already done
9735 already done
9736 already done
9737 already done
9738 already done
9739 already done
9740 already done
9741 already done
9742 already done
9743 already done
9744 already done
9745 already done
9746 already done
9747 already done
9748 already done
9749 already done
9750 already done
9751 already done
9752 already done
9753 already done
9754 already done
9755 already done
9756 already done
9757 already done
9758 already done
9759 already done
9760 already done
9761 already done
9762 already done
9763 already done
9764 already done
9765 already done
9766 already done
9767 already done
9768 already done
9769 already done
9770 already done
9771 already done
9772 already done
9773 already done
9774 already done
9775 already done
97

In [3]:
print(len(error_list))
pickle.dump(error_list, open('err4', 'wb'))

353
